In [2]:
import json
import requests

In [58]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [82]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            nresults = len(child_response['fields']['data']['message']['results'])
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return j

In [14]:
def printjson(j):
    print(json.dumps(j,indent=4))

In [15]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [16]:
def one_hop_message(subject_category,object_category,predicate,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            "predicate": predicate
        }
    }
    }
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [17]:
name_lookup('maple')

{
    "CHEBI:67940": [
        "maple furanone"
    ],
    "MONDO:0004584": [
        "maple bark disease",
        "maple bark stripper's lung",
        "maple bark-strippers' lung",
        "maple-bark strippers' lung",
        "maple bark stripper's disease",
        "alveolitis due to cryptostroma corticale"
    ],
    "MONDO:0009563": [
        "maple syrup urine disease",
        "maple syrup urine disease; MSUD",
        "maple syrup urine disease, type 2",
        "maple syrup urine disease, type 1A",
        "maple syrup urine disease, type 1B",
        "maple syrup urine disease, classic",
        "maple syrup urine disease, Intermediate",
        "maple syrup urine disease, intermittent",
        "maple syrup urine disease, thiamine-responsive",
        "MSUD",
        "Ketoacidaemia",
        "BCKD deficiency",
        "BCKDH deficiency",
        "branched chain ketoaciduria",
        "branched-chain ketoaciduria",
        "High blood leucine concentration",
        "Keto a

In [59]:
m = one_hop_message('biolink:Gene','biolink:Disease','biolink:gene_associated_with_condition',object_id='MONDO:0009563')

In [60]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Gene"
                },
                "b": {
                    "category": "biolink:Disease",
                    "id": "MONDO:0009563"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:gene_associated_with_condition"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [74]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=7965d03e-4028-42a4-938c-b7c2b7dd5375


In [83]:
j = retrieve_ars_results(mid)

Done
Done ara-aragorn 19
Error ara-ncats 0
Error kp-molecular 0
Done kp-genetics 0
Unknown ara-robokop 0
Done ara-explanatory 0
Done ara-unsecret 0
Unknown ara-improving 0
Done ara-bte 85
Done kp-textmining 0
Done kp-cam 0
Done ara-arax 7
